In [1]:
import pandas as pd



In [2]:
import snowflake.connector

In [3]:
con = snowflake.connector.connect(
    user='ritik',
    password='^$kAt54%Fe7P',
    account='ha24879.us-east-1',
    
    
)

In [4]:
con.cursor().execute("USE ROLE SYSADMIN")

In [5]:
sql='USE WAREHOUSE READ_WH;'

In [6]:
con.cursor().execute(sql)

In [7]:
sql_f="""SELECT b.ORGANIZATION_ID,b.id,a.EVENT,a.EVENT_TEXT,a.ORIGINAL_TIMESTAMP FROM "SEGMENT_DB"."OKR"."TRACKS" AS a
INNER JOIN "ALLY_USERS_DB"."PRODUCTION_VIEW"."USERS" AS b ON a.USER_ID=concat('user_',b.ID)"""


In [8]:
initial_df= con.cursor().execute(sql_f).fetch_pandas_all()

In [9]:
df=initial_df

In [10]:
df=df.dropna()

In [11]:
df.ORGANIZATION_ID = df.ORGANIZATION_ID.astype('int64') 

F:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [12]:
df=df.sort_values(by=['ORGANIZATION_ID'])

In [13]:
df=df.dropna()

In [14]:
len(df)

4429576

In [77]:
pf=df[df['EVENT']=='user_sign_up_successful']

In [16]:
pf=pf.sort_values(by=['ORGANIZATION_ID'])

In [78]:
pf

,ORGANIZATION_ID,ID,EVENT,EVENT_TEXT,ORIGINAL_TIMESTAMP
1211809,4,73232,user_sign_up_successful,User Sign-Up Successful,2020-02-05 14:35:55.496
2829310,4,81289,user_sign_up_successful,User Sign-Up Successful,2020-04-07 22:03:13.231
2933529,6,94211,user_sign_up_successful,User Sign-Up Successful,2020-07-09 22:02:56.034
1612610,6,94211,user_sign_up_successful,User Sign-Up Successful,2020-07-10 17:49:59.031
3376800,6,77856,user_sign_up_successful,User Sign-Up Successful,2020-03-10 00:01:15.120
...,...,...,...,...,...
3258515,11171,99202,user_sign_up_successful,User Sign-Up Successful,2020-08-06 19:59:21.931
4127434,11172,99204,user_sign_up_successful,User Sign-Up Successful,2020-08-06 21:32:45.827
4124635,11173,99212,user_sign_up_successful,User Sign-Up Successful,2020-08-07 03:08:50.554
4127431,11174,99213,user_sign_up_successful,User Sign-Up Successful,2020-08-07 04:05:57.194


In [108]:
pd.Timestamp.utcnow()-pf['ORIGINAL_TIMESTAMP'].iloc[0].tz_convert(tz = 'UTC') 

TypeError: Cannot convert tz-naive Timestamp, use tz_localize to localize

In [17]:
def myFunc(e):
  return e[0]

In [18]:
pql_1={}
for i in range(0,len(pf)):
  if pf.iloc[i]['ORGANIZATION_ID'] in pql_1.keys():
    ###arr=
    pql_1[pf.iloc[i]['ORGANIZATION_ID']].append([pf.iloc[i]['ORIGINAL_TIMESTAMP'],pf.iloc[i]['ID']])
    ###arr.sort(key=myFunc)
  else:
    pql_1[pf.iloc[i]['ORGANIZATION_ID']]=[[pf.iloc[i]['ORIGINAL_TIMESTAMP'],pf.iloc[i]['ID']]]

In [19]:
user_within_24_hrs=[]
for i in pql_1.keys():
    c=pd.to_datetime(pql_1[i][0][0])
    f=0
    for j in range(0,len(pql_1[i])):
        if ((pd.to_datetime(pql_1[i][j][0])-c).total_seconds()<=24*60*60):
           f+=1
    user_within_24_hrs.append(f)


In [20]:
for i in pql_1.keys():
    pql_1[i].sort(key=myFunc)

In [21]:
number_of_users=[]
for i in pql_1.keys():
    number_of_users.append(len(pql_1[i]))

In [22]:
for i in pql_1.keys():
    pql_1[i]=pql_1[i][0]
    

In [23]:
pql_1

{4: [Timestamp('2020-02-05 14:35:55.496000'), 73232],
 6: [Timestamp('2019-12-11 18:12:58.011000'), 54719],
 114: [Timestamp('2020-02-20 18:43:11.331000'), 76124],
 118: [Timestamp('2020-01-31 02:39:39.127000'), 72836],
 162: [Timestamp('2019-12-08 17:32:12.931000'), 53868],
 241: [Timestamp('2020-07-20 17:47:17.353000'), 96764],
 287: [Timestamp('2019-12-02 12:37:11.454000'), 53107],
 305: [Timestamp('2020-01-24 18:35:19.403000'), 71740],
 323: [Timestamp('2019-12-27 16:16:54.633000'), 61789],
 407: [Timestamp('2020-04-09 21:25:39.352000'), 81237],
 434: [Timestamp('2019-12-06 12:04:16.324000'), 53460],
 485: [Timestamp('2020-02-12 15:55:33.833000'), 74782],
 495: [Timestamp('2019-12-13 20:40:39.280000'), 55674],
 497: [Timestamp('2019-11-29 12:21:26.475000'), 53010],
 579: [Timestamp('2019-12-02 14:14:04.298000'), 53112],
 669: [Timestamp('2020-01-29 14:45:11.414000'), 72590],
 674: [Timestamp('2020-01-28 22:42:56.773000'), 72387],
 714: [Timestamp('2020-03-24 11:05:37.734000'), 7962

In [24]:
final_1=[]
for i in pql_1.keys():
    arr=[]
    arr.append(i)
    for j in pql_1[i]:
        arr.append(j)
    final_1.append(arr)
    
    
    


In [25]:
final_1

[[4, Timestamp('2020-02-05 14:35:55.496000'), 73232],
 [6, Timestamp('2019-12-11 18:12:58.011000'), 54719],
 [114, Timestamp('2020-02-20 18:43:11.331000'), 76124],
 [118, Timestamp('2020-01-31 02:39:39.127000'), 72836],
 [162, Timestamp('2019-12-08 17:32:12.931000'), 53868],
 [241, Timestamp('2020-07-20 17:47:17.353000'), 96764],
 [287, Timestamp('2019-12-02 12:37:11.454000'), 53107],
 [305, Timestamp('2020-01-24 18:35:19.403000'), 71740],
 [323, Timestamp('2019-12-27 16:16:54.633000'), 61789],
 [407, Timestamp('2020-04-09 21:25:39.352000'), 81237],
 [434, Timestamp('2019-12-06 12:04:16.324000'), 53460],
 [485, Timestamp('2020-02-12 15:55:33.833000'), 74782],
 [495, Timestamp('2019-12-13 20:40:39.280000'), 55674],
 [497, Timestamp('2019-11-29 12:21:26.475000'), 53010],
 [579, Timestamp('2019-12-02 14:14:04.298000'), 53112],
 [669, Timestamp('2020-01-29 14:45:11.414000'), 72590],
 [674, Timestamp('2020-01-28 22:42:56.773000'), 72387],
 [714, Timestamp('2020-03-24 11:05:37.734000'), 7962

In [26]:
pql_df_1 = pd.DataFrame(final_1, columns =['organization_id','sign_up_timestamp','first_user_sign_up'])

In [27]:
pql_df_1['number_of_users_within_life_time']=number_of_users

In [28]:
pql_df_1['users_within_24_hrs']=user_within_24_hrs

In [29]:
signup={}
for i in range(0,len(final_1)):
    signup[final_1[i][0]]=final_1[i][1]


In [30]:
sql_w="""SELECT b.ORGANIZATION_ID,c.NAME,b.id,a.EVENT,a.EVENT_TEXT,a.ORIGINAL_TIMESTAMP FROM "SEGMENT_DB"."OKR"."TRACKS" AS a
INNER JOIN "ALLY_USERS_DB"."PRODUCTION_VIEW"."USERS" AS b ON a.USER_ID=concat('user_',b.ID)
INNER JOIN "ALLY_USERS_DB"."PRODUCTION_VIEW"."ORGANIZATIONS" AS c on b.ORGANIZATION_ID=c.ID
WHERE  DATEDIFF(HOUR,a.ORIGINAL_TIMESTAMP,convert_timezone('Africa/Abidjan', current_timestamp()))<=24 AND DATEDIFF(HOUR,a.ORIGINAL_TIMESTAMP,CURRENT_TIMESTAMP)>=0;
"""

In [31]:
len(signup)

5649

In [32]:
qf= con.cursor().execute(sql_w).fetch_pandas_all()

In [33]:
signed_up_orgs=qf.loc[qf['ORGANIZATION_ID'].isin(signup.keys())]

In [67]:
len(signed_up_orgs)

22156

In [72]:
lk=signed_up_orgs.sort_values(by=['ORIGINAL_TIMESTAMP'])

In [73]:
lk

,ORGANIZATION_ID,NAME,ID,EVENT,EVENT_TEXT,ORIGINAL_TIMESTAMP
1484,11160,X,99167,viewed_confirm_trial,Viewed Confirm Trial,2020-08-06 09:00:03.434
5369,1294,F-Secure,72724,viewed_okrs,Viewed OKRs,2020-08-06 09:00:07.586
2567,1294,F-Secure,72724,viewed_okrs,Viewed OKRs,2020-08-06 09:00:11.423
14187,1294,F-Secure,72724,viewed_okrs,Viewed OKRs,2020-08-06 09:00:12.354
14798,11160,X,99167,billing_plan_changed,BillingPlan Changed,2020-08-06 09:00:14.512
...,...,...,...,...,...,...
17677,10978,deepak accnt1,98170,viewed_okrs,Viewed OKRs,2020-08-07 06:31:18.055
18693,11175,SRI Technologies,99217,setup_step_invite_users_clicked,Setup Step - Invite Users Clicked,2020-08-07 06:31:24.617
18348,1150,Urban Company,98729,objective_title_updated_from_inline_edit,Objective Title Updated From Inline Edit,2020-08-07 06:31:35.252
20056,10978,deepak accnt1,98170,viewed_okrs,Viewed OKRs,2020-08-07 06:31:59.131


In [34]:
from tqdm import tqdm

In [35]:
events={}

for i in tqdm(range(0,len(signed_up_orgs))):
    
    c=pd.to_datetime(signed_up_orgs.iloc[i]['ORIGINAL_TIMESTAMP'])-pd.to_datetime(signup[signed_up_orgs.iloc[i]['ORGANIZATION_ID']])
    if signed_up_orgs.iloc[i]['ORGANIZATION_ID'] in events.keys():
        if (c.total_seconds()<=24*60*60):
            events[signed_up_orgs.iloc[i]['ORGANIZATION_ID']].append(signed_up_orgs.iloc[i]['EVENT'])
    else:
        if(c.total_seconds()<=24*60*60):
            events[signed_up_orgs.iloc[i]['ORGANIZATION_ID']]=[signed_up_orgs.iloc[i]['EVENT']]


100%|██████████████████████████████████████████████████████████████████████████| 22156/22156 [00:16<00:00, 1349.56it/s]


In [81]:
datetime.datetime.utcnow()

datetime.datetime(2020, 8, 7, 17, 6, 20, 856686)

In [74]:
events

{11159: ['viewed_okrs',
  'viewed_okrs',
  'quick_view_opened',
  'quick_view_opened',
  'quick_search_opened_from_side_nav',
  'viewed_okrs',
  'changed_to_overview_tab_in_quick_view',
  'viewed_feed',
  'viewed_okrs',
  'viewed_feed',
  'quick_view_opened',
  'viewed_okrs',
  'changed_to_key_results_tab_in_quick_view',
  'changed_to_key_results_tab_in_quick_view',
  'changed_to_key_results_tab_in_quick_view',
  'changed_to_activity_tab_in_quick_view',
  'quick_view_opened',
  'changed_to_key_results_tab_in_quick_view',
  'viewed_okrs',
  'changed_to_overview_tab_in_quick_view',
  'quick_view_opened',
  'quick_view_opened',
  'quick_view_opened',
  'changed_to_overview_tab_in_quick_view',
  'viewed_okrs',
  'viewed_okrs',
  'changed_to_key_results_tab_in_quick_view',
  'viewed_okrs',
  'changed_to_key_results_tab_in_quick_view',
  'viewed_okrs',
  'viewed_okrs',
  'changed_to_activity_tab_in_quick_view',
  'viewed_okrs',
  'quick_view_opened',
  'changed_to_overview_tab_in_quick_view'

In [36]:
index={}

In [37]:
index['added_checkin']=0
index['added_comment']=1
index['added_objective']=2
index['average_progress_detail']=3
index['billing_invoice_current']=4
index['billing_invoice_paid']=5
index['billing_plan_changed']=6
index['browse_menu']=7
index['cancellation_request_form_submitted']=8
index['cancelled_subscription']=9
index['changed_timeperiod']=10
index['changed_to_activity_tab_in_quick_view']=11
index['changed_to_key_results_tab_in_quick_view']=12
index['changed_to_overview_tab_in_quick_view']=13
index['clicked_on_scroll_down_button_in_quick_view']=14
index['clone_objective']=15
index['closed_tour']=16
index['created_organization']=17
index['flow_completed_appcues']=18
index['flow_skipped_appcues']=19
index['flow_started_appcues']=20
index['followed_entity']=21
index['following_menu']=22
index['login_attempt_failed_inactive_subscription']=23
index['move_objective']=24
index['ms_teams_bot_installation']=25
index['ms_teams_message']=26
index['notifications_menu']=27
index['objective_description_updated_from_quick_view']=28
index['objective_detailed_view_opened_from_quick_view']=29
index['objective_entity_updated_from_quick_view']=30
index['objective_owner_updated_from_quick_view']=31
index['objective_tags_updated_from_quick_view']=32
index['objective_title_updated_from_quick_view']=33
index['quick_checked_in_okrs']=34
index['quick_view_opened']=35
index['reactivated_subscription']=36
index['removed_checkin']=37
index['removed_comment']=38
index['removed_objective']=39
index['reordered_objective']=40
index['setup_step_add_objective_clicked']=41
index['setup_step_explore_integrations_clicked']=42
index['setup_step_invite_users_clicked']=43
index['setup_step_join_policy_clicked']=44
index['setup_step_take_tour_clicked']=45
index['setup_step_watch_video_clicked']=46
index['signed_up_via_shared_link']=47
index['slack_command']=48
index['slack_interaction']=49
index['slack_pagination']=50
index['step_completed_appcues']=51
index['step_interacted_appcues']=52
index['step_skipped_appcues']=53
index['step_started_appcues']=54
index['stripe_invoice_finalized']=55
index['stripe_invoice_payment_succeeded']=56
index['subscription_cancelled']=57
index['updated_checkin']=58
index['updated_objective']=59
index['updated_objective_time_period_from_quick_view']=60
index['user_sign_in_successful']=61
index['user_sign_up_successful']=62
index['viewed_confirm_trial']=63
index['viewed_feed']=64
index['viewed_objective']=65
index['viewed_okrs']=66
index['viewed_okrs_by_status_detail']=67
index['welcome_dialog_video_ended']=68
index['welcome_dialog_video_started']=69
index['welcome_tour_completed']=70
index['welcome_tour_skipped']=71
index['welcome_tour_started']=72
index['billing_invoice_unpaid']=73
index['stripe_invoice_payment_failed']=74
index['updated_objective_weight']=75
index['inline_checkin_cancel']=76
index['inline_checkin']=77
index['quick_search_closed_with_outside_click']=78
index['objective_title_updated_from_inline_edit']=79
index['inline_checkin_initiated']=80
index['objective_owner_updated_from_inline_edit']=81
index['inline_checkin_additional_menu']=82
index['copied_link']=83
index['objective_switch_view_done']=84
index['quick_search_opened_from_side_nav']=85
index['billing_invoice_forgiven']=86
index['slack_digest_update_trigger']=87
index['email_digest_update_trigger']=88
index['viewed_okrs_from_digest_update']=89
index['objective_detailed_view_opened_from_digest_update']=90
index['manage_preference_opened_from_digest_update']=91
index['form_field_submitted_appcues']=92
index['form_submitted_appcues']=93
index['stripe_customer_subscription_trial_will_end']=94
index['updated_objective_weight_from_quick_view']=95
index['reopened_checkin']=96
index['ms_teams_task_module_fetched']=97
index['ms_teams_task_module_submitted']=98
index['un_followed_entity']=99
index['canceled_by_customer']=100
index['cancellation_averted']=101
index['objective_time_period_or_dates_updated_from_inline_edit']=102
index['objective_entity_type_updated_from_inline_edit']=103
index['completed_search']=104
index['changed_to_tasks_tab_in_quick_view']=105
index['added_project']=106
index['objective_switch_view_not_done']=107
index['started_quick_check_in']=108
index['closed_quick_check_in']=109
index['quick_search_closed_with_esc_key']=110
index['clicked_recently_visited_page_in_quick_search']=111
index['inline_add_objective_initiated']=112
index['inline_add_objective_success']=113
index['sandbox_view_accessed']=114
index['inline_add_more_options_click']=115

In [38]:
final=[]

In [39]:
pql=events

In [40]:
for i in pql.keys():
      arr1=[0]*118
      arr1[0]=i
      for j in pql[i]:
        if j in index.keys():
            arr1[index[j]+1]+=1
      arr1[117]=len(pql[i])
      final.append(arr1)
    


In [41]:
pql_df = pd.DataFrame(final, columns =['organization_id','added_checkin','added_comment','added_objective','average_progress_detail','billing_invoice_current','billing_invoice_paid','billing_plan_changed','browse_menu','cancellation_request_form_submitted','cancelled_subscription','changed_timeperiod','changed_to_activity_tab_in_quick_view','changed_to_key_results_tab_in_quick_view','changed_to_overview_tab_in_quick_view','clicked_on_scroll_down_button_in_quick_view','clone_objective','closed_tour','created_organization','flow_completed_appcues','flow_skipped_appcues','flow_started_appcues','followed_entity','following_menu','login_attempt_failed_inactive_subscription','move_objective','ms_teams_bot_installation','ms_teams_message','notifications_menu','objective_description_updated_from_quick_view','objective_detailed_view_opened_from_quick_view','objective_entity_updated_from_quick_view','objective_owner_updated_from_quick_view','objective_tags_updated_from_quick_view','objective_title_updated_from_quick_view','quick_checked_in_okrs','quick_view_opened','reactivated_subscription','removed_checkin','removed_comment','removed_objective','reordered_objective','setup_step_add_objective_clicked','setup_step_explore_integrations_clicked','setup_step_invite_users_clicked','setup_step_join_policy_clicked','setup_step_take_tour_clicked','setup_step_watch_video_clicked','signed_up_via_shared_link','slack_command','slack_interaction','slack_pagination','step_completed_appcues','step_interacted_appcues','step_skipped_appcues','step_started_appcues','stripe_invoice_finalized','stripe_invoice_payment_succeeded','subscription_cancelled','updated_checkin','updated_objective','updated_objective_time_period_from_quick_view','user_sign_in_successful','user_sign_up_successful','viewed_confirm_trial','viewed_feed','viewed_objective','viewed_okrs','viewed_okrs_by_status_detail','welcome_dialog_video_ended','welcome_dialog_video_started','welcome_tour_completed','welcome_tour_skipped','welcome_tour_started','billing_invoice_unpaid',
'stripe_invoice_payment_failed',
'updated_objective_weight',
'inline_checkin_cancel',
'inline_checkin',
'quick_search_closed_with_outside_click',
'objective_title_updated_from_inline_edit',
'inline_checkin_initiated',
'objective_owner_updated_from_inline_edit',
'inline_checkin_additional_menu',
'copied_link',
'objective_switch_view_done',
'quick_search_opened_from_side_nav',
'billing_invoice_forgiven',
'slack_digest_update_trigger',
'email_digest_update_trigger',
'viewed_okrs_from_digest_update',
'objective_detailed_view_opened_from_digest_update',
'manage_preference_opened_from_digest_update',
'form_field_submitted_appcues',
'form_submitted_appcues',
'stripe_customer_subscription_trial_will_end',
'updated_objective_weight_from_quick_view',
'reopened_checkin',
'ms_teams_task_module_fetched',
'ms_teams_task_module_submitted',
'un_followed_entity',
'canceled_by_customer',
'cancellation_averted',
'objective_time_period_or_dates_updated_from_inline_edit',
'objective_entity_type_updated_from_inline_edit',
'completed_search',
'changed_to_tasks_tab_in_quick_view',
'added_project',
'objective_switch_view_not_done',
'started_quick_check_in',
'closed_quick_check_in',
'quick_search_closed_with_esc_key',
'clicked_recently_visited_page_in_quick_search',
'inline_add_objective_initiated',
'inline_add_objective_success',
'sandbox_view_accessed',
'inline_add_more_options_click',
                                      'Total_EVENTS_COUNTS'])


In [79]:
pql_df

,organization_id,added_checkin,added_comment,added_objective,average_progress_detail,billing_invoice_current,billing_invoice_paid,billing_plan_changed,browse_menu,cancellation_request_form_submitted,...,objective_switch_view_not_done,started_quick_check_in,closed_quick_check_in,quick_search_closed_with_esc_key,clicked_recently_visited_page_in_quick_search,inline_add_objective_initiated,inline_add_objective_success,sandbox_view_accessed,inline_add_more_options_click,Total_EVENTS_COUNTS
0,11159,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,47
1,11164,2,0,4,0,0,0,1,0,0,...,0,0,0,0,0,5,3,0,2,73
2,11170,1,0,21,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,77
3,11167,0,0,11,0,0,0,1,0,0,...,0,0,0,0,0,2,1,0,0,55
4,11149,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,3,3,0,0,60
5,11160,0,1,5,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,43
6,11165,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,8
7,11168,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,36
8,11166,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,6
9,11157,0,0,9,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,28


In [42]:
result = pd.merge(pql_df,
                 pql_df_1,
                 on='organization_id')

In [43]:
qu=result['organization_id'].values
rs=[]
for i in range(0,len(qu)):
    rs.append("'"+'organization_'+str(qu[i])+"'")
qur=','.join([i for i in rs]) 

In [44]:
sql='''SELECT DISTINCT ORG_ID,PAID_CUSTOMER FROM "ALLY_USERS_DB"."ENGINEERING"."ACCOUNT_LEVEL_VIEW"
WHERE ORG_ID IN ({qur});'''.format(qur=qur)

In [45]:
signups= con.cursor().execute(sql).fetch_pandas_all()

In [46]:
signups=signups.rename(columns={"ORG_ID": "organization_id"})

In [47]:

for i in range(0,len(signups)):
    signups.iloc[i]['organization_id']=signups.iloc[i]['organization_id'].strip('organization_')

In [48]:
signups = signups.astype('int64') 

In [86]:
timedelta(days = 1) 

NameError: name 'timedelta' is not defined

In [49]:
result_final = pd.merge(result,
                 signups,
                 on='organization_id')

In [50]:
ispql=[0]*len(result_final)
for i in range(0,len(result_final)):
    if result_final.iloc[i]['Total_EVENTS_COUNTS']>=25:
        ispql[i]=1
    else:
        ispql[i]=0
    

In [51]:
from datetime import date

today = date.today()
d4 = today.strftime("%b-%d-%Y")
import datetime
y=str(datetime.datetime.now().time())


In [52]:
hr=y[0:2]
min=y[3:5]
time=hr+'-'+min

In [53]:
result_final['pql?']=ispql

In [54]:
result_final=result_final.sort_values(by=['organization_id'])

In [55]:
result_final=result_final.reset_index()

In [119]:
pd.Timestamp.utcnow().tz_localize(None)

Timestamp('2020-08-07 18:03:31.111509')

In [121]:
(pd.Timestamp.utcnow().tz_localize(None)-result_final['sign_up_timestamp'].iloc[0]).total_seconds()

186367.72514

In [128]:
for i in range(0,len(result_final)):
    d=(pd.Timestamp.utcnow().tz_localize(None)-result_final['sign_up_timestamp'].iloc[0]).total_seconds()
    if d>24*60*60:
        result_final.drop(result_final.index[i])


In [129]:
result_final

,index,organization_id,added_checkin,added_comment,added_objective,average_progress_detail,billing_invoice_current,billing_invoice_paid,billing_plan_changed,browse_menu,...,inline_add_objective_success,sandbox_view_accessed,inline_add_more_options_click,Total_EVENTS_COUNTS,sign_up_timestamp,first_user_sign_up,number_of_users_within_life_time,users_within_24_hrs,PAID_CUSTOMER,pql?
0,15,11145,0,0,0,0,0,0,0,0,...,0,0,0,3,2020-08-05 14:17:50.872,99042,5,5,0,0
1,4,11149,1,0,0,0,0,0,0,0,...,3,0,0,60,2020-08-05 15:03:49.904,99051,1,1,0,1
2,9,11157,0,0,9,0,0,0,0,0,...,0,0,0,28,2020-08-05 21:28:49.915,99127,2,2,0,1
3,0,11159,0,0,0,0,0,0,0,0,...,0,0,0,47,2020-08-06 08:35:15.931,99165,1,1,0,1
4,5,11160,0,1,5,0,0,0,1,0,...,0,0,0,43,2020-08-06 08:59:57.149,99167,1,1,0,1
5,14,11161,0,0,0,0,0,0,1,0,...,0,0,0,8,2020-08-06 11:36:26.485,99171,1,1,0,0
6,10,11162,0,0,1,0,0,0,1,0,...,0,0,0,6,2020-08-06 12:52:35.775,99175,1,1,0,0
7,13,11163,0,0,0,0,0,0,1,0,...,0,0,0,12,2020-08-06 13:21:35.163,99177,1,1,0,0
8,1,11164,2,0,4,0,0,0,1,0,...,3,0,2,73,2020-08-06 13:21:59.224,99178,2,2,0,1
9,6,11165,0,0,0,0,0,0,1,0,...,0,0,0,8,2020-08-06 13:28:42.178,99180,1,1,0,0


In [57]:
sql_w="""SELECT ID,NAME FROM "ALLY_USERS_DB"."PRODUCTION_VIEW"."ORGANIZATIONS";"""

In [58]:
orgs_with_id= con.cursor().execute(sql_w).fetch_pandas_all()

In [59]:
orgs_with_id

,ID,NAME
0,10695,treses
1,10692,ofri internet
2,10691,Friendfinder
3,9711,The BAD Marketing
4,10689,Waterloo Sparkling Water
...,...,...
10641,8854,Tucows Inc.
10642,3886,Indeed Sandbox
10643,4483,PowerHouse Consulting Group
10644,5887,Accolade


In [60]:
orgs_with_id=orgs_with_id.rename(columns={"ID": "organization_id"})

In [61]:
result_final_1 = pd.merge(result_final,
                 orgs_with_id,
                 on='organization_id')

In [159]:
extracted=result_final_1[['organization_id','NAME','Total_EVENTS_COUNTS','sign_up_timestamp','pql?','PAID_CUSTOMER']]

In [160]:
extracted

,organization_id,NAME,Total_EVENTS_COUNTS,sign_up_timestamp,pql?,PAID_CUSTOMER
0,11145,BioSYLx,3,2020-08-05 14:17:50.872,0,0
1,11149,Test,60,2020-08-05 15:03:49.904,1,0
2,11157,Core Spaces,28,2020-08-05 21:28:49.915,1,0
3,11159,sjcompany,47,2020-08-06 08:35:15.931,1,0
4,11160,X,43,2020-08-06 08:59:57.149,1,0
5,11161,EMILIANO VELASQUEZ,8,2020-08-06 11:36:26.485,0,0
6,11162,Muhammed,6,2020-08-06 12:52:35.775,0,0
7,11163,Spark Info,12,2020-08-06 13:21:35.163,0,0
8,11164,Thoughtss Incorporation,73,2020-08-06 13:21:59.224,1,0
9,11165,Headway,8,2020-08-06 13:28:42.178,0,0


In [161]:
extracted=extracted.rename(columns={'NAME':'organization_name','Total_EVENTS_COUNTS':'Total_events_count','sign_up_timestamp':'trial_start','PAID_CUSTOMER':'paid?'})

In [162]:
extracted

,organization_id,organization_name,Total_events_count,trial_start,pql?,paid?
0,11145,BioSYLx,3,2020-08-05 14:17:50.872,0,0
1,11149,Test,60,2020-08-05 15:03:49.904,1,0
2,11157,Core Spaces,28,2020-08-05 21:28:49.915,1,0
3,11159,sjcompany,47,2020-08-06 08:35:15.931,1,0
4,11160,X,43,2020-08-06 08:59:57.149,1,0
5,11161,EMILIANO VELASQUEZ,8,2020-08-06 11:36:26.485,0,0
6,11162,Muhammed,6,2020-08-06 12:52:35.775,0,0
7,11163,Spark Info,12,2020-08-06 13:21:35.163,0,0
8,11164,Thoughtss Incorporation,73,2020-08-06 13:21:59.224,1,0
9,11165,Headway,8,2020-08-06 13:28:42.178,0,0


In [167]:
pu=[]
for i in range(0,len(extracted)):
    d=(pd.Timestamp.utcnow().tz_localize(None)-extracted['trial_start'].iloc[i]).total_seconds()
    if d>24*60*60:
        pu.append(i)
        


In [168]:
(pd.Timestamp.utcnow().tz_localize(None)-extracted['trial_start'].iloc[i]).total_seconds()

44660.212896

In [169]:
extracted=extracted.drop(extracted.index[pu])

In [170]:
extracted

,organization_id,organization_name,Total_events_count,trial_start,pql?,paid?
15,11171,Fly - MarTech Made Easy,28,2020-08-06 19:59:21.931,1,0
16,11172,Test1234,10,2020-08-06 21:32:45.827,0,0
17,11173,kris,120,2020-08-07 03:08:50.554,1,0
18,11174,Test123,4,2020-08-07 04:05:57.194,0,0
19,11175,SRI Technologies,21,2020-08-07 06:06:22.188,0,0


In [66]:
extracted.to_excel(r'F:\pql_final\orgs_within_24hrs_'+str(d4)+'_'+time+'.xlsx')